In [ ]:
#Name --> Nikhil,  
#Roll No --> 3434
#Assignment 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X = dataset.iloc[:,3:13].values 
Y = dataset.iloc[:,-1].values

In [4]:
#Encode categorical values - geography, gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [5]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [7]:
#Standardize the features
X = pipeline.fit_transform(X)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [10]:
#Initialize ANN
classifier = Sequential()

In [11]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [12]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [13]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [14]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [15]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [16]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 1s - loss: 0.5934 - accuracy: 0.7258 - val_loss: 0.4990 - val_accuracy: 0.7950 - 1s/epoch - 5ms/step
Epoch 2/200
225/225 - 0s - loss: 0.4875 - accuracy: 0.7961 - val_loss: 0.4526 - val_accuracy: 0.7987 - 300ms/epoch - 1ms/step
Epoch 3/200
225/225 - 0s - loss: 0.4580 - accuracy: 0.8043 - val_loss: 0.4278 - val_accuracy: 0.8075 - 299ms/epoch - 1ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4339 - accuracy: 0.8169 - val_loss: 0.4106 - val_accuracy: 0.8200 - 302ms/epoch - 1ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4223 - accuracy: 0.8217 - val_loss: 0.3985 - val_accuracy: 0.8288 - 288ms/epoch - 1ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4163 - accuracy: 0.8233 - val_loss: 0.3905 - val_accuracy: 0.8338 - 300ms/epoch - 1ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4047 - accuracy: 0.8297 - val_loss: 0.3833 - val_accuracy: 0.8325 - 292ms/epoch - 1ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4011 - accuracy: 0.8325 - val_loss: 0.3767 - val_accuracy: 0.8363 - 302ms/epoch - 1ms/

In [17]:
y_pred = classifier.predict(X_test)
y_pred

array([[0.27570623],
       [0.3182181 ],
       [0.24106482],
       ...,
       [0.1913741 ],
       [0.16880354],
       [0.23922223]], dtype=float32)

In [18]:
# Converting in Binary Values
y_pred = (y_pred>0.5).astype(int)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [19]:
#Printing Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1543,   52],
       [ 219,  186]], dtype=int64)

In [20]:
#Printing Accuracy Score
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.8645